In [ ]:
from __future__ import annotations
import os
import sys
import json
from datetime import timedelta
from importlib import reload
from aws_lambda_powertools import Logger
from aws_lambda_powertools.event_handler import Response, content_types
from requests import status_codes
from code.handler_lambda.src.helpers.network import APIS, make_request, RequestResponse
from code.handler_lambda.src.helpers.datetime import jenkins_build_datetime
from code.handler_lambda.src.calculators.shared import (
    FiveHundredError,
    extract_parent_commits,
    fetch_parent_commit_statuses,
    get_last_build_of_parent_commit,
    get_first_jenkins_build_of_current_pull_request,
    get_at_jenkins_build_of_current_pull_request,
    get_pr_jenkins_build_of_current_pull_request,
    JenkinsHistoryLimit
)

BITBUCKET_WORKSPACE = os.getenv("BITBUCKET_WORKSPACE", "workspace")
BITBUCKET_REPO_SLUG = os.getenv("BITBUCKET_REPO_SLUG", "repo")

JENKINS_AT_JOB_NAME = os.getenv("JENKINS_AT_JOB_NAME", "job")
JENKINS_PR_JOB_NAME = os.getenv("JENKINS_PR_JOB_NAME", "job")

logger = Logger()



In [ ]:
for module_key in [key for key in sys.modules.keys() if "code.handler_lambda" in key]:
    reload(sys.modules[module_key])


In [ ]:
from code.handler_lambda.src.helpers.network import APIS, make_request, RequestResponse
from code.handler_lambda.src.helpers.datetime import jenkins_build_datetime
from code.handler_lambda.src.calculators.shared import (
    FiveHundredError,
    extract_parent_commits,
    fetch_parent_commit_statuses,
    get_last_build_of_parent_commit,
    get_first_jenkins_build_of_current_pull_request,
    get_at_jenkins_build_of_current_pull_request,
    get_pr_jenkins_build_of_current_pull_request,
    JenkinsHistoryLimit
)


In [ ]:
def get_num_of_pull_requests():
    num_of_pull_requests_request_url = f"/repositories/{BITBUCKET_WORKSPACE}/{BITBUCKET_REPO_SLUG}/pullrequests?state=MERGED&fields=size"

    num_of_bitbucket_pull_requests_response = make_request(
        APIS.BITBUCKET, num_of_pull_requests_request_url
    )

    if not num_of_bitbucket_pull_requests_response["success"]:
        logger.error(
            "bitbucket request errored out",
            url=num_of_pull_requests_request_url,
            response=num_of_bitbucket_pull_requests_response,
        )
        raise FiveHundredError(response=num_of_bitbucket_pull_requests_response)

    logger.info(
        "successfully got the number of pull requests",
        response=num_of_bitbucket_pull_requests_response,
    )

    try:
        num_of_bitbucket_pull_requests = num_of_bitbucket_pull_requests_response[
            "data"
        ]["size"]
    except KeyError as err:
        raise FiveHundredError(f"Key {str(err)} cannot be found in the dict")

    return num_of_bitbucket_pull_requests


def get_all_pull_requests(number_of_pull_requests):
    pagelen = min(50, number_of_pull_requests)
    all_pull_requests_url = f"/repositories/{BITBUCKET_WORKSPACE}/{BITBUCKET_REPO_SLUG}/pullrequests?state=MERGED&pagelen={pagelen}&fields=values.source.branch,values.id,values.title,values.state,values.merge_commit.hash,values.merge_commit.date,values.merge_commit.links.self.href,values.merge_commit.links.statuses.href,values.merge_commit.parents,values.merge_commit.parents.hash,values.merge_commit.parents.date,values.merge_commit.parents.links.self.href,values.merge_commit.parents.links.html.href,values.merge_commit.parents.links.statuses.href"

    all_pull_request_response = make_request(APIS.BITBUCKET, all_pull_requests_url)

    if not all_pull_request_response["success"]:
        logger.error(
            "bitbucket request errored out",
            url=all_pull_requests_url,
            response=all_pull_request_response,
        )
        raise FiveHundredError(response=all_pull_request_response)

    logger.info(
        "successfully got all of the pull requests",
        response=all_pull_request_response,
    )

    return all_pull_request_response["data"]


In [ ]:
def get_timestamp_of_pr_build_of_pull_request(pull_request):
    parent_commit_hash, parent_commit_hash_url, statuses_of_parent_commit_url = extract_parent_commits(pull_request)

    last_build_of_parent_commit_display_url = fetch_parent_commit_statuses(parent_commit_hash, parent_commit_hash_url, statuses_of_parent_commit_url)

    if "master" in last_build_of_parent_commit_display_url:
        raise JenkinsHistoryLimit()

    first_jenkins_build_of_current_pull_request_url = get_last_build_of_parent_commit(last_build_of_parent_commit_display_url)

    first_jenkins_build_of_current_pull_request_id, _ = get_first_jenkins_build_of_current_pull_request(first_jenkins_build_of_current_pull_request_url)

    first_jenkins_at_build_of_current_pull_request_id = get_at_jenkins_build_of_current_pull_request(first_jenkins_build_of_current_pull_request_id)

    first_jenkins_pr_build_of_current_pull_request_duration_seconds, first_jenkins_pr_build_of_current_pull_request_start_timestamp = get_pr_jenkins_build_of_current_pull_request(first_jenkins_at_build_of_current_pull_request_id)

    first_jenkins_pr_build_of_current_pull_request_finish_timestamp = (
        first_jenkins_pr_build_of_current_pull_request_start_timestamp
        + first_jenkins_pr_build_of_current_pull_request_duration_seconds
    )

    return first_jenkins_pr_build_of_current_pull_request_finish_timestamp


def filter_only_hotfix_pull_requests(pull_requests):
    max_pull_requests_count = len(pull_requests) + 1
    filtered_pull_request_indexes = []

    for index, pull_request in enumerate(pull_requests):
        try:
            if index < max_pull_requests_count and "hotfix" in pull_request["source"]["branch"]["name"]:
                filtered_pull_request_indexes.append(index)
        except KeyError as err:
            raise FiveHundredError(message=f"Key {str(err)} cannot be found in the dict")

    return filtered_pull_request_indexes


def filter_out_hotfix_pull_requests(pull_requests):
    filtered_pull_request_indexes = filter_only_hotfix_pull_requests(pull_requests)

    inserted_count = 0
    filtered_pull_request_with_non_hotfixes = []
    for i, pull_request_index in enumerate(filtered_pull_request_indexes):
        higher_pull_request_index = pull_request_index + 1
        filtered_pull_request_with_non_hotfixes.append(
            pull_requests[pull_request_index]
        )
        if (i < (len(filtered_pull_request_indexes) - 1)):
            if (filtered_pull_request_indexes[i + 1] != higher_pull_request_index):
                filtered_pull_request_with_non_hotfixes.append(
                    pull_requests[higher_pull_request_index]
                )
                inserted_count += 1
        else:
            if (pull_request_index < len(pull_requests)):
                filtered_pull_request_with_non_hotfixes.append(
                    pull_requests[higher_pull_request_index]
                )

    return filtered_pull_request_with_non_hotfixes


# main program !!!
def get_time_to_restore_service():
    num_of_bitbucket_pull_requests = get_num_of_pull_requests()

    pull_requests_response = get_all_pull_requests(num_of_bitbucket_pull_requests)

    pull_requests = []

    try:
        pull_requests = pull_requests_response["values"]
    except KeyError as err:
        raise FiveHundredError(message=f"Key {str(err)} cannot be found in the dict")

    filtered_pull_request_with_non_hotfixes = filter_out_hotfix_pull_requests(pull_requests)

    time_to_recoverys = []

    for pull_request in filtered_pull_request_with_non_hotfixes:
        try:
            jenkins_pr_build_of_current_pull_request_finish_timestamp = get_timestamp_of_pr_build_of_pull_request(pull_request)
        except JenkinsHistoryLimit:
            break

        jenkins_pr_build_of_current_pull_request_finish_datetime = jenkins_build_datetime(
            {"timestamp": jenkins_pr_build_of_current_pull_request_finish_timestamp}
        )

        if pull_request == filtered_pull_request_with_non_hotfixes[0]:
            finish_datetime_one = jenkins_pr_build_of_current_pull_request_finish_datetime
            continue
        
        finish_datetime_two = jenkins_pr_build_of_current_pull_request_finish_datetime

        duration: timedelta = (
            finish_datetime_one
            - finish_datetime_two
        )

        time_to_recoverys.append(duration.total_seconds())

        finish_datetime_one = finish_datetime_two

    mean_time_to_recovery_seconds = sum(time_to_recoverys) / len(
        time_to_recoverys
    )

    mean_time_to_recovery_timedelta = timedelta(seconds=mean_time_to_recovery_seconds)

    return Response(
        status_code=status_codes.codes.OK,
        content_type=content_types.APPLICATION_JSON,
        body={
            "meanTimeToRecoverySeconds": mean_time_to_recovery_seconds,
            "meanTimeToRecoveryDurations": str(mean_time_to_recovery_timedelta).split('.')[0].replace(':', ' hr(s), ', 1).replace(':', ' min(s), ', 1) + ' sec(s)'
        },
    )




In [ ]:
try:
    finala = get_time_to_restore_service()
except FiveHundredError as err:
    finala = Response(
        status_code=status_codes.codes.SERVER_ERROR,
        content_type=content_types.APPLICATION_JSON,
        body={
            "message": err.message
        },
    )



print(f"\nHTTP Status: {finala.status_code}")
print(json.dumps(finala.body, indent=2))
